<h3>Genetic Algorithm</h3>

Genetic algorithms simulate the process of natural selection [ is a mechanism of evolution that explains how populations of living organisms adapt and change over time. ]. In simpler words, they simulate 'survival of the fittest' among individuals of consecutive generations to solve a problem.

<h5>Libraries and global variables</h5>

In [29]:
import random
import math
import logging
import time

from tqdm import trange,tqdm #visualizing the loops

GLOBAL_CHROMOSOME_LENGTH=4
GLOBAL_RANDINT_MIN=0
GLOBAL_RANDINT_MAX=8
GLOBAL_POP_LIMIT=16             #it is a X-X grid

GLOBAL_MUTATOR_CHANCE=100
GLOBAL_MUTATOR_RANGE=1000

GLOBAL_EPOCH_COUNT=4
GLOBAL_FITTING_DATA=[i for i in range(GLOBAL_CHROMOSOME_LENGTH)]

<h4>Gene Class</h4>

In [30]:
class Gene:
    if True:
        #Initializzation of stored variables
        stored_chromosome_length=0
        stored_randint_min=0            #used in mutator
        stored_randint_max=0            #used in mutator
        stored_gene_fitness_factor=9999

        stored_parent1=0
        stored_parent2=0

        #Initialization of internal lists
        chromosomes=[]

    def __init__(self,fitting_data=GLOBAL_FITTING_DATA,input_list=[],
                 chromosome_length=GLOBAL_CHROMOSOME_LENGTH,randint_min=GLOBAL_RANDINT_MIN,randint_max=GLOBAL_RANDINT_MAX,
                 gene_generator='random',evaluator='default-evaluator',initial_eval=False): #constructor
        self.chromosomes=[]
        self.stored_chromosome_length=chromosome_length
        self.stored_randint_min=randint_min
        self.stored_randint_max=randint_max
        match gene_generator: 
            case 'none':
                pass
            case 'random': #initializes the gene based on chromosome length and randint range
                for i in range(chromosome_length):
                    self.chromosomes.append(random.randint(randint_min,randint_max)) 
            case 'input':  #takes in an input list and assigns the list to the chromosomes and updates the chromosome length to match the input list
                self.chromosomes=input_list
                self.stored_chromosome_length=len(input_list)
        if initial_eval:
            try:
                self.fitness_evaluation(fitting_data,evaluator=evaluator)
            except:
                pass
    def crossover_with_gene(self,input_gene, #crosses over with another gene, uniform crossover, includes mutation
                            crossover_method='cartesian-prod-random-uniform',
                            mutate=True): 
        child=Gene(chromosome_length=self.stored_chromosome_length,gene_generator='')
        match crossover_method:
            case 'cartesian-prod-random-uniform': #50-50 gamba on every chromosome 
                for i in range(self.stored_chromosome_length):
                    if bool(random.getrandbits(1)):
                        child.chromosomes.append(self.chromosomes[i])
                    else:
                        child.chromosomes.append(input_gene.chromosomes[i])
            case 'equate-cartesian-prod-random-uniform':
                pass
            case 'average-integer':
                for i in range(self.stored_chromosome_length):
                    child.chromosomes.append(int((self.chromosomes[i]+input_gene.chromosomes[i])/2))
        child.random_mutator(mutate=mutate)
        return child
    def random_equation_go_brrr(self,input_gene, #BRRRRRR
                                equation_target='minimize'): 
        equated_value=0
        match equation_target:
            case 'minimize':
                
                int(self.chromosomes[0])-int()

    def random_mutator(self,mutate=True): #imitates mutation based on mutation chances 
        if mutate:
            if random.randint(1,GLOBAL_MUTATOR_RANGE)<=GLOBAL_MUTATOR_CHANCE:
                self.chromosomes[random.randint(0,self.stored_chromosome_length-1)]=random.randint(self.stored_randint_min,self.stored_randint_max)
    def fitness_evaluation(self,fitting_data, #generates the fitness factor based on fitting_data
                           evaluator='default-evaluator'): 
        match evaluator:
            case 'default-evaluator':
                self.stored_gene_fitness_factor=self.stored_chromosome_length
                for i in range(self.stored_chromosome_length):
                    if self.chromosomes[i]==fitting_data[i]:
                        self.stored_gene_fitness_factor-=1
        return self.stored_gene_fitness_factor
    def display(self): #returns a string with the chromosomes and fitness factor as output
        return str('Chromosomes: '+str(self.chromosomes)+' Fitness Factor: '+str(self.stored_gene_fitness_factor))


<h4>Gene List Class</h4>
<p>It is a 2D Square [AxA] matrix comprised of 'Gene's.

### Bug List
---
- #1 [Minor] In Gene_List_Eval, the parameter stop=False does not work

- #2 [Minor] stored_generation is 1 higher than what it should be (?)

- #3 [Major] stored_variables in Gene_List class have instancing issues 


#### #3
##### Related to python's inherent behaviour regarding variables, https://stackoverflow.com/questions/986006/how-do-i-pass-a-variable-by-reference
##### Only fixable by using mutable objects such as lists, good to know for the future.

---

> ``` python
> GLOBAL_POP_LIMIT=16
> class Gene_List:
>    stored_pop_limit=0
>    def __init__(self,pop_limit=GLOBAL_POP_LIMIT):
>       self.stored_pop_limit=pop_limit
>       self.Gene_List_Gen()
>    def Gene_List_Gen(self,pop_limit=stored_pop_limit):
>        print(pop_limit)
> A=Gene_list(pop_limit=12)
> ```

> ``` python
> 0
> ```

> ``` python
> GLOBAL_POP_LIMIT=16
> class Gene_List:
>     stored_pop_limit=[0]
>     def __init__(self,pop_limit=GLOBAL_POP_LIMIT):
>         self.stored_pop_limit.clear()
>         self.stored_pop_limit.append(pop_limit)
>         self.Gene_List_Gen()
>     def Gene_List_Gen(self,pop_limit=stored_pop_limit):
>         print(pop_limit)
> A=Gene_List(pop_limit=12)
> ```

> ``` python
> [12]
> ```

##### To-Do List 

generators are stable so you procrastinating little shit go work on actually doing a simple fucking addition in an equation\
re: wtf am i doing



In [ ]:
class Gene_List: 
    if True:
        #Initialization of the logger
        logger=logging.getLogger
        
        #Initialization of stored variables
        stored_epoch_count=GLOBAL_EPOCH_COUNT
        stored_current_epoch=0
        stored_pop_limit=1
        stored_chromosome_length=GLOBAL_CHROMOSOME_LENGTH
        stored_randint_min=GLOBAL_RANDINT_MIN
        stored_randint_max=GLOBAL_RANDINT_MAX
        stored_generation=0
        stored_logger_level=logging.DEBUG

        #Initialization of preferred settings
        preferred_gene_generator='random'
        preferred_crossover_method='cartesian-prod-random-uniform'
        preferred_crossover_prune_method='random-survivor'
        preferred_evaluator='default-evaluator'

        #Initialization of boolean variables
        bool_stop=True
        bool_verbose=False
        bool_mutate=True
        bool_initial_eval=False
        bool_manual=False

        bool_pcheck_safe=True

        #Initialization of internal lists
        fitting_data=[]
        generation_list=[]
        index_list=[]
        gene_list=[]
        
    def __init__(self,fitting_data=GLOBAL_FITTING_DATA,epoch=GLOBAL_EPOCH_COUNT,pop_limit=GLOBAL_POP_LIMIT,
                 chromosome_length=GLOBAL_CHROMOSOME_LENGTH,randint_min=GLOBAL_RANDINT_MIN,randint_max=GLOBAL_RANDINT_MAX,
                 gene_generator=preferred_gene_generator,crossover_method=preferred_crossover_method,crossover_prune_method=preferred_crossover_prune_method,evaluator=preferred_evaluator,
                 stop=bool_stop,verbose=bool_verbose,mutate=bool_mutate,initial_eval=bool_initial_eval,manual=bool_manual):
        #Store Input Parameters
        self.logger=self.Generate_Logger()
        self.stored_epoch_count=epoch
        self.stored_pop_limit=pop_limit
        self.stored_chromosome_length=chromosome_length
        self.stored_randint_min=randint_min
        self.stored_randint_max=randint_max

        self.preferred_gene_generator=gene_generator
        self.preferred_crossover_method=crossover_method
        self.preferred_crossover_prune_method=crossover_prune_method
        self.preferred_evaluator=evaluator

        self.bool_stop=stop
        self.bool_verbose=verbose
        self.bool_mutate=mutate
        self.bool_initial_eval=initial_eval
        self.bool_manual=manual

        self.fitting_data=fitting_data
        self.abcd=0
        #Emptying Generated Lists
        self.generation_list=[]
        self.index_list=[]

        self.Parameter_Checks()
        
        if self.bool_pcheck_safe:
            self.gene_list=self.Gene_List_Gen(fitting_data=self.fitting_data,pop_limit=pop_limit,
                                              chromosome_length=self.stored_chromosome_length,randint_min=self.stored_randint_min,randint_max=self.stored_randint_max,
                                              gene_generator=self.preferred_gene_generator,evaluator=self.preferred_evaluator,
                                              initial_eval=self.bool_initial_eval)
            self.generation_list.append(self.gene_list)     #appends the initial gene list to the generation list, it is generation zero
            if manual==False:
                self.List_Iterator()
        else:
            self.logger.error('Parameter check error.')
    def Generate_Logger(self): #Creates a logger for use
        logging.basicConfig(filename='Gen_Algo.log',format='%(asctime)s - %(name)s - %(levelname)s: %(message)s',
                            datefmt='%Y-%m-%d %H:%M:%S',filemode='w',level=self.stored_logger_level)
        logger=logging.getLogger()
        logger.info('Logger initialized.')
        return logger
    def Assign_Variables(self,pop_limit): #[Deprecated]
        self.stored_pop_limit=pop_limit
    def Parameter_Checks(self): #Checks Input Parameters for errors
        if (self.stored_randint_max-self.stored_randint_min+1)<self.stored_chromosome_length:
            self.logger.warning('Chromosome length greater than random range.')
            self.bool_pcheck_safe=False
        if self.stored_chromosome_length==0:
            self.logger.warning('Chromosome length set to zero.')
    def List_Iterator(self): #Auto Iterator
        self.logger.info('List Iterator called')
        self.stored_current_epoch=0
        self.logger.debug('Epoch counter reset')
        for i in trange(self.stored_epoch_count,desc='Epoch'):
            if self.index_list==[] or self.bool_stop==False:                                                            #if no fit gene is found
                self.generation_list.append(self.Gene_List_Cross(input_list=self.generation_list[i],pop_limit=self.stored_pop_limit,
                                                                 crossover_method=self.preferred_crossover_method,crossover_prune_method=self.preferred_crossover_prune_method,
                                                                 verbose=self.bool_verbose,mutate=self.bool_mutate))    #generate new generation
                self.stored_generation+=1
                self.Gene_List_Eval(self.generation_list[i],evaluator=self.preferred_evaluator,verbose=self.bool_verbose,stop=self.bool_stop)  
                self.stored_current_epoch+=1                      #evaluate the new generation
            if self.index_list!=[] and self.bool_stop==True:                                                         #if fit gene found
                self.Gene_List_Viewer(self.generation_list[i],self.bool_verbose)                           #display fit gene
                return                                                                     #end loop
    def Gene_List_Gen(self,fitting_data=fitting_data,input_list=[],pop_limit=stored_pop_limit, #Gene List Generator [Initial] [Standalone+Iterator]
                      chromosome_length=stored_chromosome_length,randint_min=stored_randint_min,randint_max=stored_randint_max,
                      gene_generator=preferred_gene_generator,evaluator=preferred_evaluator,
                      initial_eval=bool_initial_eval): 
        result=[]
        temp=[]
        self.logger.info('Generating gene list with pop_limit='+str(pop_limit)+
                         ', chromosome_length='+str(chromosome_length)+', randint_min='+str(randint_min)+', randint_max='+str(randint_max)+
                         ', gene_generator='+gene_generator+', evaluator='+evaluator+', initial_eval='+str(initial_eval))
        match gene_generator:
            case 'random':
                for i in trange(pop_limit,desc='Generating initial genes [Random]'):
                    temp=[]
                    for j in range(pop_limit):
                        temp.append(Gene(fitting_data=fitting_data,
                                        chromosome_length=chromosome_length,randint_min=randint_min,randint_max=randint_max,
                                        gene_generator=gene_generator,evaluator=evaluator,
                                        initial_eval=initial_eval))
                    result.append(temp)
            case 'sequence':
                print('Work in Progress')
        self.logger.debug('Generation completed.')
        return result
    def Gene_List_Cross(self,input_list,pop_limit=stored_pop_limit, #Gene List Crossover Function [Standalone+Iterator]
                        crossover_method=preferred_crossover_method,crossover_prune_method=preferred_crossover_prune_method,
                        verbose=bool_verbose,mutate=bool_mutate): 
        result=[]
        cross=[]
        temp=[]
        self.logger.info('Crossover using '+crossover_method+' on epoch '+str(self.stored_current_epoch)+' with pop_limit '+str(pop_limit)+' whereas stored is '+str(self.stored_pop_limit))
        if verbose:
            match crossover_method:
                case 'cartesian-prod-random-uniform':
                    ct_prod_vb='Crossover function executing [Cartesian]'
                    for i in trange(len(input_list),desc=ct_prod_vb):
                        for j in range(len(input_list)):
                            temp=[]
                            for k in range(len(input_list)):
                                for l in range(len(input_list)):
                                    temp.append(input_list[i][j].crossover_with_gene(input_list[k][l],mutate=mutate,crossover_method=crossover_method))
                            cross.append(temp)
                case 'equate-cartesian-prod-random-uniform': #deprecating
                    eq_prod_vb='Crossover function executing [Cartesian Equate]'
                    for i in trange(len(input_list),desc=eq_prod_vb):
                        for j in range(len(input_list)):
                            temp=[]
                            for k in range(len(input_list)):
                                for l in range(len(input_list)):
                                    temp.append(input_list[i][j].crossover_with_gene(input_list[k][l],mutate=mutate,crossover_method=crossover_method))
                            cross.append(temp)
                case 'average-integer':
                    avg_int_vb='Crossover function executing [Average]'
                    for i in trange(len(input_list),desc=avg_int_vb):
                        for j in range(len(input_list)):
                            temp=[]
                            for k in range(len(input_list)):
                                for l in range(len(input_list)):
                                    temp.append(input_list[i][j].crossover_with_gene(input_list[k][l],mutate=mutate,crossover_method=crossover_method))
                            cross.append(temp)
            match crossover_prune_method:
                case 'random-survivor':
                    rand_verbose='Selecting random survivors'
                    surv_list=[]
                    for i in trange(pop_limit,desc=rand_verbose):
                        temp=[]
                        for j in range(pop_limit):
                            surv_x=random.randint(0,math.pow(pop_limit,2)-1)
                            surv_y=random.randint(0,math.pow(pop_limit,2)-1)
                            temp.append(cross[surv_x][surv_y])
                        surv_list.append(temp)
                    result=surv_list
            return result
        else:
            match crossover_method:
                case 'cartesian-prod-random-uniform':
                    for i in range(len(input_list)):
                        for j in range(len(input_list)):
                            temp=[]
                            for k in range(len(input_list)):
                                for l in range(len(input_list)):
                                    temp.append(input_list[i][j].crossover_with_gene(input_list[k][l],mutate=mutate,crossover_method=crossover_method))
                            cross.append(temp)
                case 'equate-cartesian-prod-random-uniform': #deprecating
                    for i in range(len(input_list)):
                        for j in range(len(input_list)):
                            temp=[]
                            for k in range(len(input_list)):
                                for l in range(len(input_list)):
                                    temp.append(input_list[i][j].crossover_with_gene(input_list[k][l],mutate=mutate,crossover_method=crossover_method))
                            cross.append(temp)
                case 'average-integer':
                    for i in range(len(input_list)):
                        for j in range(len(input_list)):
                            temp=[]
                            for k in range(len(input_list)):
                                for l in range(len(input_list)):
                                    temp.append(input_list[i][j].crossover_with_gene(input_list[k][l],mutate=mutate,crossover_method=crossover_method))
                            cross.append(temp)
            match crossover_prune_method:
                case 'random-survivor':
                    surv_list=[]
                    for i in range(pop_limit):
                        temp=[]
                        for j in range(pop_limit):
                            surv_x=random.randint(0,math.pow(pop_limit,2)-1)
                            surv_y=random.randint(0,math.pow(pop_limit,2)-1)
                            temp.append(cross[surv_x][surv_y])
                        surv_list.append(temp)
                    result=surv_list
            return result
    def Gene_List_Eval(self,input_list_1,input_list_2=[], #Gene List Fitness Evaluation Function [Standalone+Iterator]
                       evaluator='default-evaluator',
                       verbose=True,stop=False): 
        result=[]
        temp=[]
        self.logger.info('Evaluating with '+evaluator+' on epoch '+str(self.stored_current_epoch))
        match evaluator:
            case 'default-evaluator':
                if verbose:
                    eval_str='Evaluating genes [Default Evaluator]'
                    for i in trange(len(input_list_1),desc=eval_str):
                        for j in range(len(input_list_1[0])):
                            if input_list_1[i][j].fitness_evaluation(self.fitting_data)==0:
                                temp=[]
                                temp.append(i)
                                temp.append(j)
                                result.append(temp)
                                if stop:
                                    self.index_list=result
                                    return
                    self.index_list=result
                    pass
                else:
                    for i in range(len(input_list_1)):
                        for j in range(len(input_list_1[0])):
                            if input_list_1[i][j].fitness_evaluation(self.fitting_data)==0:
                                temp=[]
                                temp.append(i)
                                temp.append(j)
                                result.append(temp)
                                if stop:
                                    self.index_list=result
                                    return
                    self.index_list=result
                    pass
            case 'maximize':
                max=[[0],[],[]]
                if verbose:
                    eval_str='Evaluating genes [Maximize]'
                    for i in trange(len(input_list_1)):
                        for j in range(len(input_list_1)):
                            for k in range(len(input_list_2)):
                                for l in range(len(input_list_2)):
                                    if max[0][0]<(input_list_1[i][j].chromosomes[0]+input_list_2[k][l].chromosomes[0]):
                                        max[1]=[]
                                        max[2]=[]
                                        max[1].append(i)
                                        max[1].append(j)
                                        max[2].append(k)
                                        max[2].append(l)
                                        max[0][0]=input_list_1[i][j].chromosomes[0]+input_list_2[k][l].chromosomes[0]
                else:
                    for i in range(len(input_list_1)):
                        for j in range(len(input_list_1)):
                            for k in range(len(input_list_2)):
                                for l in range(len(input_list_2)):
                                    if max[0][0]<(input_list_1[i][j].chromosomes[0]+input_list_2[k][l].chromosomes[0]):
                                        max[1]=[]
                                        max[2]=[]
                                        max[1].append(i)
                                        max[1].append(j)
                                        max[2].append(k)
                                        max[2].append(l)
                                        max[0][0]=input_list_1[i][j].chromosomes[0]+input_list_2[k][l].chromosomes[0]
                    return max
    def Gene_List_Viewer(self,input_list,verbose=True): #Gene List Viewer [Standalone+Iterator Version]
        if verbose:
            for i in tqdm(self.index_list,desc='Displaying genes'):
                print('Gene '+str(i)+' '+input_list[i[0]][i[1]].display())
        else:
            for i in self.index_list:
                print('Gene '+str(i)+' '+input_list[i[0]][i[1]].display())

In [32]:
X=Gene_List(pop_limit=10,chromosome_length=1,randint_min=1,randint_max=10,gene_generator='random',mutate=False,manual=True)
Y=Gene_List(pop_limit=3,chromosome_length=1,randint_min=-10,randint_max=10,gene_generator='random',mutate=False,manual=True)

Generating initial genes [Random]: 100%|██████████| 3/3 [00:00<?, ?it/s]


In [33]:
X.stored_pop_limit

10

In [34]:
X.Gene_List_Gen()

Generating initial genes [Random]: 100%|██████████| 1/1 [00:00<?, ?it/s]


[[<__main__.Gene at 0x235188c14d0>]]

In [35]:
X.Gene_List_Eval(X.generation_list[0],Y.generation_list[0],evaluator='maximize',verbose=False)

[[20], [1, 6], [1, 2]]

In [36]:
# GList3=Gene_List(mutate=False,manual=True,gene_generator='none')

# GList3.generation_list.append(GList1.Gene_List_Cross(GList2.generation_list[0],crossover_method='average-integer'))
# GList3.generation_list

In [37]:
GList4=Gene_List(epoch=1000,stop=True,pop_limit=12)

Epoch:   3%|▎         | 28/1000 [00:02<01:17, 12.61it/s]

Gene [8, 11] Chromosomes: [0, 1, 2, 3] Fitness Factor: 0


In [38]:
GList4.stored_generation

29

<h6><i>Everything below is deprecated</i></h6>

In [39]:
def GLG(): #Gene List Generator [Initial]
    result=[]
    temp=[]
    for i in trange(GLOBAL_POP_LIMIT,desc='Generating genes'):
        temp=[]
        for j in range(GLOBAL_POP_LIMIT):
            temp.append(Gene(rand_gene=True))
        result.append(temp)
    return result

In [40]:
def GLCF(input_list): #Gene List Crossover Function
    result=[]
    temp=[]
    for i in trange(len(input_list),desc='Crossover function executing'):
        for j in range(len(input_list)):
            temp=[]
            for k in range(len(input_list)):
                for l in range(len(input_list)):
                    temp.append(input_list[i][j].crossover_with_gene(input_list[k][l]))
            result.append(temp)
    return result

In [41]:
def GLFEF(input_list,fitting_data,stop=False): #Gene List Fitness Evaluation Function
    result=[]
    temp=[]
    for i in trange(len(input_list),desc='Evaluating genes'):
        for j in range(len(input_list[0])):
            if input_list[i][j].fitness_evaluation(fitting_data)==0:
                temp=[]
                temp.append(i)
                temp.append(j)
                result.append(temp)
                if stop:
                    return result
    return result

In [42]:
def Gene_List_Viewer(input_list,index_list): #Gene List Viewer
    for i in tqdm(index_list,desc='Displaying genes'):
        print('Gene '+str(i)+' '+input_list[i[0]][i[1]].display())

In [43]:
def GCLRS(input_list): #Gene Crossover List Random Sampler
    len_list=len(input_list)
    parent_1=random.randint(0,len_list-1)
    parent_2=random.randint(0,len_list-1)
    print('\nParent 1 ['+str(parent_1)+']:'+input_list[parent_1][parent_1].display())
    print('\nParent 2 ['+str(parent_2)+']:'+input_list[parent_1][parent_1].display())
    print('\nChild 1 (P1xP2): '+input_list[parent_1][parent_2].display())
    print('\nChild 2 (P2xP1): '+input_list[parent_2][parent_1].display())

In [44]:
fitting_data=[x for x in range(GLOBAL_CHROMOSOME_LENGTH)]
list_of_genes=GLG()
first_gen=GLCF(list_of_genes)
index_list=GLFEF(first_gen,fitting_data=fitting_data,stop=True)
Gene_List_Viewer(first_gen,index_list)

Generating genes:   0%|          | 0/16 [00:00<?, ?it/s]


TypeError: Gene.__init__() got an unexpected keyword argument 'rand_gene'

In [ ]:
run=True
gene_list=GLG()
generation_list=[]
generation_list.append(gene_list)
index_list=[]
while run==False:
    for i in trange(GLOBAL_EPOCH_COUNT):
        if index_list==[]:
            generation_list.append(GLCF(generation_list[i]))
            index_list=GLFEF(generation_list[i],fitting_data,stop=True)
        if index_list!=[]:
            Gene_List_Viewer(generation_list[i],index_list)
            run=False
            break
    run=False

In [ ]:
GCLRS(first_gen)

In [ ]:
class Gene_ID_Gen: #deprecated
    gene_id_tracker=0
    def generate(self):
        gene_id=self.gene_id_tracker+1
        self.gene_id_tracker=self.gene_id_tracker+1
        return gene_id
    def latest(self):
        return self.gene_id_tracker

ID_Gen=Gene_ID_Gen()